## connect

In [1]:
import pandas as pd
import sqlite3

/Users/a19028558/Desktop/transfer_accounts/akovyaz/learning_myself/env_school/lib/python3.8/site-packages/pandas/compat/__init__.py:97: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
connect = sqlite3.connect('../data/checking-logs.sqlite')

## schema

In [3]:
pd.io.sql.read_sql("PRAGMA table_info(test);", con=connect)

,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,uid,TEXT,0,None,0
2,2,labname,TEXT,0,None,0
3,3,first_commit_ts,TIMESTAMP,0,None,0
4,4,first_view_ts,TIMESTAMP,0,None,0


## first 10

In [4]:
pd.io.sql.read_sql("select * from test limit 10;", con=connect)

,index,uid,labname,first_commit_ts,first_view_ts
0,0,user_17,project1,2020-04-18 07:56:45,2020-04-18 10:56:55
1,1,user_30,laba04,2020-04-18 13:36:53,2020-04-17 22:46:26
2,2,user_30,laba04s,2020-04-18 14:51:37,2020-04-17 22:46:26
3,3,user_14,laba04,2020-04-18 15:14:00,2020-04-18 10:53:52
4,4,user_14,laba04s,2020-04-18 22:30:30,2020-04-18 10:53:52
5,5,user_19,laba04,2020-04-20 19:05:01,2020-04-21 20:30:38
6,6,user_25,laba04,2020-04-20 19:16:50,2020-05-09 23:54:54
7,7,user_21,laba04,2020-04-21 17:48:00,2020-04-22 22:40:36
8,8,user_30,project1,2020-04-22 12:36:24,2020-04-17 22:46:26
9,9,user_21,laba04s,2020-04-22 20:09:21,2020-04-22 22:40:36


## Min

In [5]:
df_min = pd.io.sql.read_sql(
    """select uid, cast(min(deadlines - cast(strftime('%s', first_commit_ts) as int))/3600 as int) as delta
    from test join deadlines on labs = labname where labs != 'project1';""",
    con=connect)
df_min

,uid,delta
0,user_25,2


## Max

In [6]:
df_max = pd.io.sql.read_sql(
    """select uid, cast(max(deadlines - cast(strftime('%s', first_commit_ts) as int))/3600 as int) as delta
    from test join deadlines on labs = labname where labs != 'project1';""",
    con=connect)
df_max

,uid,delta
0,user_30,202


## Avg

In [7]:
df_avg = pd.io.sql.read_sql(
    """select cast(avg(deadlines - cast(strftime('%s', first_commit_ts) as int))/3600 as int) as delta
    from test join deadlines on labs = labname where labs != 'project1';""",
    con=connect)
df_avg

,delta
0,89


## Corr

In [8]:
views_diff = pd.io.sql.read_sql(
    """select test.uid, cast(avg(deadlines - cast(strftime('%s', first_commit_ts) as int))/3600 as int) as avg_diff,
    count as pageviews from test join deadlines on labs = labname
    left join (select uid, count(uid) as count from pageviews group by uid) t on t.uid = test.uid
    where labs != 'project1' group by test.uid; """,
    con=connect)

In [9]:
views_diff.corr()

,avg_diff,pageviews
avg_diff,1.000000,0.276022
pageviews,0.276022,1.000000


## Close connect

In [10]:
connect.close()